In [74]:
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2
import json
import matplotlib.pyplot as plt
import os

In [75]:
# image file folder
FILE_PATH = ['datasets/300W-Style/300W-Gray/', 'datasets/300W-Style/300W-Light/',
            'datasets/300W-Style/300W-Original/', 'datasets/300W-Style/300W-Sketch/']
# testset position
TEST_PATH = ['helen/testset', 'lfpw/testset', 'ibug']
# use 68 key points face model
SHAPE_PREDICTOR = "../../LipMotionDetection/model/shape_predictor_68_face_landmarks.dat"
# define lip region
(LIPFROM, LIPTO) = (49, 68)
# define threshold for lip motion
THRESHOLD = 0.65

In [76]:
# define the face detector
DETECTOR = dlib.get_frontal_face_detector()
# define a shape predictor
PREDICTOR = dlib.shape_predictor(SHAPE_PREDICTOR)

In [77]:
TEST_PATH_COMMON = []
TEST_PATH_CHALLENGING = []
for file in FILE_PATH:
    TEST_PATH_COMMON.append(file + TEST_PATH[0])
    TEST_PATH_COMMON.append(file + TEST_PATH[1])
    TEST_PATH_CHALLENGING.append(file + TEST_PATH[2])
TEST_PATH_FULL = TEST_PATH_COMMON + TEST_PATH_CHALLENGING

In [78]:
def getFileList(dir, ext=None):
    Filelist = []        
    if os.path.isdir(dir):
        for f in os.listdir(dir):
            if ext is None:
                Filelist.append(path + '/' + f)
            elif ext in f[-3:]:
                    Filelist.append(path + '/' + f)
 
    return Filelist

In [79]:
def loadGroundTruth(imgFile):
    prefix = imgFile[:-4]
    ptsFile = prefix +'.pts'
    return np.loadtxt(ptsFile, comments=("version:", "n_points:", "{", "}"))

In [80]:
IMAGE_COMMON = []
for path in TEST_PATH_COMMON:
    temp = getFileList(dir=path, ext='jpg')
    if len(temp) == 0:
        temp = getFileList(dir=path, ext='png')
    IMAGE_COMMON += temp
IMAGE_CHALLENGING = []
for path in TEST_PATH_CHALLENGING:
    temp = getFileList(dir=path, ext='jpg')
    if len(temp) == 0:
        temp = getFileList(dir=path, ext='png')
    IMAGE_CHALLENGING += temp
IMAGE_FULL = IMAGE_COMMON + IMAGE_CHALLENGING

In [81]:
def getDisP2P(a, b):
    return np.linalg.norm(np.array(a)-np.array(b))

In [82]:
def getMeanDisF2F(A, B):
    Dis = 0
    for i in range(len(A)):
        dis = getDisP2P(A[i], B[i])
        Dis += dis
    # print('Mean Dis:', Dis/len(A))
    return Dis/len(A)

In [86]:
def getDio(face):
    left = [np.mean(face[36:42][0]), np.mean(face[36:42][1])]
    right = [np.mean(face[42:48][0]), np.mean(face[42:48][1])]
    # print(left, right)
    Dio = getDisP2P(left, right)
    # print('Dio:', Dio)
    return Dio

In [84]:
def getNME(A, B):
    NME = getMeanDisF2F(A, B)/getDio(B)
    return NME

In [100]:
NME_COMMON_SUM = 0
MISS_COMMON = 0
for idx, path in enumerate(IMAGE_COMMON):
    temp = []
    temp_GT = []
    image=cv2.imread(path)
    # plt.imshow(image)
    # plt.show()
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = DETECTOR(image_gray, 0)
 
    if len(rects):
        nme = -1
        for rect in rects:
            shape = PREDICTOR(image_gray, rect)
            shape = face_utils.shape_to_np(shape)
            temp = shape
            temp_GT = loadGroundTruth(path)
            if (nme == -1):
                nme = getNME(temp, temp_GT)
            else:
                nme = min(nme, getNME(temp, temp_GT))
        NME_COMMON_SUM += nme
    else:
        MISS_COMMON += 1
    if(idx % 100 == 0):
        print(idx, '/', len(IMAGE_COMMON))

NME_COMMON = NME_COMMON_SUM / (len(IMAGE_COMMON)-MISS_COMMON)

0 / 2216
100 / 2216
200 / 2216
300 / 2216
400 / 2216
500 / 2216
600 / 2216
700 / 2216
800 / 2216
900 / 2216
1000 / 2216
1100 / 2216
1200 / 2216
1300 / 2216
1400 / 2216
1500 / 2216
1600 / 2216
1700 / 2216
1800 / 2216
1900 / 2216
2000 / 2216
2100 / 2216
2200 / 2216


In [101]:
print(MISS_COMMON)
print(1-MISS_COMMON/len(IMAGE_COMMON))
print(NME_COMMON)

47
0.9787906137184116
0.07697584362504409


In [108]:
NME_CHALLENGING_SUM = 0
MISS_CHALLENGING = 0
FALSE_CHALLENGING = 0
for idx, path in enumerate(IMAGE_CHALLENGING):
    temp = []
    temp_GT = []
    image=cv2.imread(path)
    # plt.imshow(image)
    # plt.show()
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = DETECTOR(image_gray, 0)

    if len(rects):
        nme = -1
        for rect in rects:
            shape = PREDICTOR(image_gray, rect)
            shape = face_utils.shape_to_np(shape)
            temp = shape
            temp_GT = loadGroundTruth(path)
            if (nme == -1):
                nme = getNME(temp, temp_GT)
            else:
                nme = min(nme, getNME(temp, temp_GT))
        NME_CHALLENGING_SUM += nme
        if (nme > 0.3):
            FALSE_CHALLENGING += 1
    else:
        MISS_CHALLENGING += 1
    if(idx % 100 == 0):
        print(idx, '/', len(IMAGE_CHALLENGING))

NME_CHALLENGING = NME_CHALLENGING_SUM / (len(IMAGE_CHALLENGING)-MISS_CHALLENGING)

0 / 540
100 / 540
200 / 540
300 / 540
400 / 540
500 / 540


In [109]:
print(MISS_CHALLENGING)
print(1-MISS_CHALLENGING/len(IMAGE_CHALLENGING))
print(FALSE_CHALLENGING)
print(1-(MISS_CHALLENGING+FALSE_CHALLENGING)/len(IMAGE_CHALLENGING))
print(NME_CHALLENGING)

116
0.7851851851851852
82
0.6333333333333333
2.2675725895714125


In [104]:
MISS_FULL = MISS_COMMON + MISS_CHALLENGING
print(MISS_FULL)
print(1-MISS_FULL/len(IMAGE_FULL))

NME_FULL_SUM = NME_COMMON_SUM + NME_CHALLENGING_SUM
NME_FULL = NME_FULL_SUM / (len(IMAGE_FULL)-MISS_FULL)
print(NME_FULL)

163
0.940856313497823
0.43517600570806
